In [5]:
# initialization
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# importing Qiskit
from qiskit import IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

# import basic plot tools
from qiskit.tools.visualization import plot_histogram

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends() 

In [4]:
for backend in provider.backends():
    print(backend.status())

In [3]:
import csv

Fidelity=[]
dimension=0
Trials=8192
counter=0

Repetitions=50 #Set the number of repetitions of the experiment (8192 shots preferred) per chip per number of qubits 
QSet=[1,2,3,4,5] #Represent the set of the number of qubits to be evaluated

#QuantumChips=['ibmqx2','ibmq_16_melbourne','ibmq_vigo','ibmq_ourense','ibmq_london','ibmq_burlington','ibmq_essex']  #Full set of chips
QuantumChips=['ibmq_burlington','ibmq_essex','ibmq_london']#'ibmq_london']#,'ibmq_essex'] #Partial set of chips

Generalkeys=['Run', '# of qubits', 'Chip', '# of shots', 'Fidelity', 'Runtime (ms)', 'Job ID']

for n in QSet:
    qubits=n
    Possibleresults=[]
    for num in range(2**n):
        Possibleresults.append(bin(num)[2:].zfill(n))
    Possibleresults.reverse()

    wstring=[]
    for ky in Possibleresults:
        wstring.append(str(ky))
    
    Partnkeys=Generalkeys+wstring
    
    import csv
    
    nameN='Results'+str(qubits)+'.csv'
    with open(nameN,'w',newline='') as newFile:
        newFileWriter = csv.writer(newFile)
        newFileWriter.writerow(Partnkeys)
    
        
            
    for chip in QuantumChips:
     
        for k in range (0,Repetitions):
            Km=k
            Stfidel=''
            dimension=n
            for j in range (n):
                Stfidel=Stfidel+'1' 
    
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
    
            djCircuit = QuantumCircuit(qr, cr)
            barriers = True

            # Apply barrier 
            if barriers:
                djCircuit.barrier()

            # Apply Hadamard gate
            djCircuit.h(qr)    
    
            # Apply barrier 
            if barriers:
                djCircuit.barrier()
    
            # Apply Z-gate 
            djCircuit.z(qr)  

            # Apply barrier 
            if barriers:
                djCircuit.barrier()

            # Apply Hadamard gate
            djCircuit.h(qr) 

            # Measure the register
    
            for i in range(n):
                djCircuit.measure(qr[i], cr[i])
        
            real_device = provider.get_backend(chip)
            #real_device = provider.get_backend('ibmq_burlington')
            job_exp = execute(djCircuit, backend=real_device, shots=Trials, memory=True)
   
            exp_result = job_exp.result()

            exp_measurement_result = exp_result.get_counts(djCircuit)
        
            counter=counter+1
        
            Fidelity.append(exp_measurement_result[Stfidel]/Trials)
            for i in Possibleresults:
                if i not in exp_measurement_result:
                    exp_measurement_result.update({i: 0})
            Ordic={}
            for j in Possibleresults:
                Ordic.update({j:exp_measurement_result.get(j,0)})
            Quandiop={}
            Quandiop={"Run":counter,"# of qubits":qubits,"Chip": str(real_device),"# of shots":Trials,"Fidelity": exp_measurement_result[Stfidel]/Trials,"Runtime (ms)":exp_result.time_taken}
            Quandiop.update({"Job ID":job_exp.job_id()})
            Quandiop.update(Ordic)
            lia=[]
            for j in Quandiop.values():
                lia.append(j)
            #with open('Results.csv', 'w', newline='') as file:
            #    writer.writerow(lia)
            print(Quandiop) #Manual verification of the process
            with open(nameN, 'a', newline='') as newFile:
                newFileWriter = csv.writer(newFile)
                newFileWriter.writerow(lia)